In [43]:
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import UnstructuredExcelLoader
from pymongo import MongoClient
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
import os

# 1. 엑셀 파일 로드 및 문서로 변환
xlsx_path = 'test용사전엑셀.xlsx'
xlsx_loader = UnstructuredExcelLoader(xlsx_path, mode="elements")
docs = xlsx_loader.load()  # 로드된 원본 문서

# 2. Embeddings 및 문서 분할 설정
embeddings = OpenAIEmbeddings()
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in docs])




In [49]:
from pymongo import MongoClient

# 3. MongoDB 연결 설정
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "test"
COLLECTION_NAME = "col"
# INDEX_NAME = "_id_"

collection = client[DB_NAME][COLLECTION_NAME]


vectorstore = MongoDBAtlasVectorSearch.from_documents(
    documents=semantic_chunks,  # **chunker로 분할된 문서 사용**
    embedding=embeddings,      # Embedding 모델
    collection=collection,      # MongoDB 컬렉션
    # index_name=INDEX_NAME,
)


In [46]:
# 번역을 위한 PromptTemplate 설정
translation_prompt = PromptTemplate(
    input_variables=["retrieved_text", "query"],
    template="""
다음은 한국어 텍스트와 관련된 참고 자료입니다:
참고 자료: {retrieved_text}

위 자료를 기반으로, 아래 한국어 문장을 영어로 번역해주세요.
문장: {query}
번역:
"""
)

trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

translation_chain = LLMChain(
    llm=trans_llm, 
    prompt=translation_prompt
    )

# 키워드 매칭, 점수를 내서 처리
retriever = vectorstore.as_retriever()


In [47]:
query = "산독끼는 바나나와 꼬라니를 좋아해."
retrieved_docs = retriever.get_relevant_documents(query)

print("지금 뭐가 있는데?", retrieved_docs )

# 검색 결과를 결합하여 참고 자료로 활용
retrieved_text = " ".join([doc.page_content for doc in retrieved_docs])

result = translation_chain.run({"retrieved_text": retrieved_text, "query": query})
print("번역 결과:", result)

지금 뭐가 있는데? []
번역 결과: The translation of the sentence "산독끼는 바나나와 꼬라니를 좋아해." is "Sandoqki likes bananas and corani."
